In [1]:
import os
import sys
import obspy
from obspy.taup import TauPyModel
import glob
import subprocess

init_dir = os.getcwd()

def sort_and_add(fnames, gc_list, az_list):
    newfnames = []
    for fname in glob.glob(fnames):
        
        gcmin, gcmax = min(gc_list), max(gc_list)
        azmin, azmax = min(az_list), max(az_list)
        
        tr = obspy.read(fname)[0]

        gc, az = tr.stats.sac['gcarc'], tr.stats.sac['az']
        if (gc >= gcmin and gc <= gcmax) and (az >= azmin and az <= azmax):
            newfnames.append(fname)
    
    return newfnames

def align_on_(model, phase):
    taup_sac = False
    if phase == 'P':
        talign = '-T+t1'
        tlabel = '\"Time [s] aligned on P\"'
    elif phase == 'S':
        talign = '-T+t2'
        tlabel = '\"Time [s] aligned on S\"'
    elif phase == 'Strue':
        talign = '-T+t3'
        tlabel = '\"Time [s] aligned on S true\"'
    elif phase == 'o':
        talign = '-T+t-3'
        tlabel = '\"Time [s] from event\"'
    elif phase == 'PKIKP':
        talign = '-T+t4'
        tlabel = '\"Time [s] aligned on PKIKP\"'
    elif phase == 'r':
        talign = '-T+r3.8'
        tlabel = '\"Time-distance/3.8(km/s) [s]\"'
    elif phase == 'l':
        talign = '-T+r4.4'
        tlabel = '\"Time-distance/4.4(km/s) [s]\"'
    elif type(phase) == float:
        talign = '-T+r%f' % phase
        tlabel = '\"Time-distance/%f(km/s) [s]\"' % phase
    else:
        taup_sac = True
        talign = '-T+t5'
        tlabel = '\"Time [s] aligned on %s\"' % phase
    
    return tlabel, taup_sac, talign

def seis_data_plot(dirname, fnames, scale, gcarc_list, azimuth_list, 
                   time_list, align_phase, gmtcmd, gmt_begin=True, 
                   gmt_end=True, annotation=True, align_model='prem', 
                   pen_color='black', outfile=None):
    if os.path.exists(dirname):
        os.chdir(dirname)
    else:
        sys.exit('No such dir: %s!' % dirname)
    
    subfnames = sort_and_add(fnames=fnames, gc_list=gcarc_list, 
                             az_list=azimuth_list)
    # option '-T' in gmt sac
    tlabel, taup_sac, talign = align_on_(model=align_model, phase=align_phase)
    # option '-C' in gmt sac
    tmin, tmax, ta = time_list
    clabel = '-C%s/%s' % (tmin, tmax)
    # option '-M' in gmt sac
    mlabel = '-M%s' % scale
    # option '-R' in gmt sac
    gcmin, gcmax = min(gcarc_list) - 0.1, max(gcarc_list) + 0.1
    rlabel = '-R%s/%s/%s/%s' % (tmin, tmax, gcmin, gcmax)
    # option '-B' in gmt sac
    blabel = '-Bx%s+l%s -Bya5f1+l\"Distance [deg]\" -BWSen' % (ta, tlabel)
    # outfile name
    if not outfile:
        outfname = '%s.az.%s.%s.gc.%s.%s' % (fnames[-1], min(azimuth_list), 
                                             max(azimuth_list), min(gcarc_list), 
                                             max(gcarc_list))
    else:
        outfname = outfile
    
    if gmt_begin:
        gmtcmd += 'gmt begin %s pdf\n' % outfname
    gmtcmd += 'gmt basemap -JX14c/21c %s %s\n' % (rlabel, blabel)
    for fname in subfnames:
        gmtcmd += 'gmt sac %s %s %s %s -Ed -W0.5p,%s\n' % (fname, talign, 
                                                           clabel, mlabel, pen_color)
        tr = obspy.read(fname)[0]
        gc, knetwk = tr.stats.sac['gcarc'], tr.stats.sac['knetwk']
        az, kstnm = tr.stats.sac['az'], tr.stats.sac['kstnm']
        if annotation:
            gmtcmd += 'echo %s %s %s %s %.2f | gmt text -F+f8p,6,black+a0+jML -N\n' \
                % (tmax*1.01, gc, knetwk, kstnm, az)
    if annotation and subfnames:
        h = (tmin + tmax)*0.5
        gmtcmd += 'echo %s %s %s Az = %s~%s deg. Delta = %s~%s deg. | \
            gmt text -F+f12p,6,black+a0+jMC -D0c/0.7c -N\n' \
                % (h, gcmax, fnames[-1], min(azimuth_list), max(azimuth_list), 
                   min(gcarc_list), max(gcarc_list))
        gmtcmd += 'echo %s %s %s %s, lat = %.2f, lon = %.2f, depth = %.2f | \
            gmt text -F+f10p,6,black+a0+jMC -D0c/1.2c -N\n' \
                % (h, gcmax, tr.stats.sac['nzyear'], tr.stats.sac['nzjday'], 
                   tr.stats.sac['evla'], tr.stats.sac['evlo'], tr.stats.sac['evdp'])
    gmtcmd += 'echo \"0 %s\n0 %s\" | gmt plot -W1p,red,-\n' % (gcmin, gcmax)
    
    if gmt_end:
        gmtcmd += 'gmt end show'
    
    return gmtcmd

def reverse_R_to_T(dirname):
    if os.path.exists(dirname):
        os.chdir(dirname)
    else:
        sys.exit('No such dir: %s!' % dirname)
    mul = -1.0
    for fname in glob.glob('*.BHR'):
        T = '.'.join(fname.split('.')[:-1]) + '.BHT'
        if os.path.exists(T):
            trT = obspy.read(T)[0]
            trR = obspy.read(fname)[0]
            
            trR.data = trR.data * mul
            R = '.'.join(fname.split('.')[:-1]) + '.R'
            trR.write(R, format='SAC')
        else:
            print('rm %s due to no %s' % (fname, T))
            os.unlink(fname)
    
    return

In [3]:
gmt_cmd = 'export GMT_SESSION_NAME=$$\n'

# necessary params
dirname = '/home/sheng/桌面/seis/alaska/20200906/sac1'
gc_list = [64, 92]
az_list = [24, 22]
compare = False
t_list = [-50, 150, 'a50f10']
scale = '0.6c'
outfile = 'az.%s.%s.gc.%s.%s.TR' % (min(az_list), max(az_list), 
                                   min(gc_list), max(gc_list))
align_phase = 'S'

#reverse_R_to_T(dirname=dirname)

if not compare:
    gmt_cmd = seis_data_plot(dirname=dirname, fnames='*.BHT', scale=scale, 
                             gcarc_list=gc_list, azimuth_list=az_list, 
                             time_list=t_list, align_phase=align_phase, 
                             gmtcmd=gmt_cmd, gmt_begin=True, gmt_end=True, 
                             annotation=True, align_model='prem', 
                             pen_color='black')
    
else:
    gmt_cmd = seis_data_plot(dirname=dirname, fnames='*.BHT', scale=scale, 
                             gcarc_list=gc_list, azimuth_list=az_list, 
                             time_list=t_list, align_phase=align_phase, 
                             gmtcmd=gmt_cmd, gmt_begin=True, gmt_end=False, 
                             annotation=True, align_model='prem', 
                             pen_color='black', outfile=outfile)
    
    gmt_cmd = seis_data_plot(dirname=dirname, fnames='*.R', scale=scale, 
                             gcarc_list=gc_list, azimuth_list=az_list, 
                             time_list=t_list, align_phase=align_phase, 
                             gmtcmd=gmt_cmd, gmt_begin=False, gmt_end=True, 
                             annotation=False, align_model='prem', 
                             pen_color='red', outfile=outfile)
    




subprocess.Popen(['bash'], stdin=subprocess.PIPE).communicate(gmt_cmd.encode())

os.chdir(init_dir)

In [26]:
a = []
if a:
    print(1)